In [1]:
import numpy as np
import matplotlib.pyplot as plt
from inspect import signature
from itertools import product
from functools import reduce
from collections import Counter
import time

def get_ddt(sbox):
    """
    Returns the Difference Distribution Table (DDT) for `sbox`
    """
    l = len(sbox)
    ddt = np.zeros((l,l), dtype=int)
    for i,x in enumerate(sbox):
        for j,y in enumerate(sbox):
            ddt[i^j][x^y] += 1
    return ddt

def tobits(x:int, nbits:int): 
    """
    Unpacks `x` into a list of `nbits` bits in little endian
    """
    return [*map(int, format(x, "0%db"%nbits)[::-1])]

def frombits(bits):
    """
    Packs `b`, a list of bits into an integer in little endian
    """
    return sum(b*(1<<i) for i,b in enumerate(bits))

In [8]:
import random

def swap(a,x,y):
    a[x] ^= a[y]
    a[y] ^= a[x]
    a[x] ^= a[y]

random.seed(1)
PERM = np.array([0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51,
        4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55,
        8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59,
        12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63], dtype=np.uint64)

#random.shuffle(SBOX := [*range(16)])
SBOX = [(3 + 7*j)%16 for j in range(16)]
len(set(SBOX))
swap(SBOX, 5, 2)
swap(SBOX, 14, 10)

In [9]:
# Precompiled

opt = True
threshold = 4

# SBOX H-rep
ddt = get_ddt(SBOX)
ddt_thres = ddt > threshold
d = dict([(x,y) for x in range(16) for y in range(16) if ddt_thres[x,y] != 0])
DDT_SBOX = np.array([d[i] for i in range(16)], dtype=np.uint64)

In [11]:
from numba import jit, uint64, byte, types, float64

@jit(uint64(byte[:]))
def byte2int(blk):
    r = 0
    for i in range(8):
        r += blk[i] << (i*8)
    return r

@jit(byte[:](uint64))
def int2byte(x):
    r = np.zeros(8, dtype=np.uint8)
    for i in range(8): r[i] = (x >> (i*8))&0xff
    return r

@jit(uint64[:](byte[:]))
def toints(pt):
    r = np.zeros(len(pt)//8, dtype=np.uint64)
    for i in range(len(pt)//8): r[i] = byte2int(pt[8*i:8*i+8])
    return r

@jit(uint64(uint64))
def sub(p):
    r = 0
    for i in range(16):
        r = r | DDT_SBOX[(p >> (i*4)) & 0xf] << (i*4)
    return r

@jit(uint64(uint64))
def perm(p):
    r = 0
    for i in range(64):
        r |= ((p >> i) & 1) << PERM[i]
    return r

#def encryptblk(p, nrounds):
#    trail = []
#    for k in range(nrounds):
#        trail.append(p)
#        p = sub(p)
#        p = perm(p)
#    p = sub(p)
#    return p, trail

@jit(types.Tuple((uint64, float64))(uint64, uint64))
def encryptblk(p, nrounds):
    trailprob = 1.0
    for k in range(nrounds):
        for i in range(16):
            x = (p >> (i*4))&0xf
            trailprob *= ddt[x][DDT_SBOX[x]]/16
        p = sub(p)
        p = perm(p)
    p = sub(p)
    return p, trailprob

In [12]:
from functools import reduce

def trailprobability(trail):
    prob = 1
    for t in trail:
        nibs = [(t >> (i*4))&0xf for i in range(16)]
        for x in nibs: prob *= ddt[x][DDT_SBOX[x]]/16
    return prob

def printtrail(trail):
    for t in trail:
        print("".join(".1"[i] for i in tobits(t, 64)))
        
def getctrecovered(ctdiff):
    return set(i for i in range(16) if (ctdiff >> (i*4))&0xf != 0)

In [23]:
from itertools import product

# (onein, (ptdiff, ctdiff))
goodtrails = [None]*16
rejected = []
for k0idx in product(range(16), repeat=2):
    
    for _ptdiff in product(range(1, 16), repeat=2):
        ptdiff = 0
        for a,b in zip(k0idx, _ptdiff):
            ptdiff |= b << (4*a)
        ctdiff, prob = encryptblk(ptdiff, 12)

        ct_recovered = getctrecovered(ctdiff)
        onein = 1/prob
        for k in ct_recovered:
            kt = goodtrails[k]
            if kt is None or kt[0] > onein:
                goodtrails[k] = (onein, (ptdiff, ctdiff))
                if kt is not None: rejected.append(kt[1][0])

        #print(format(j, "02d"), "".join(".1"[i] for i in tobits(ctdiff, 64)), 1/trailprobability(trail))
    
    print(k0idx, end="\r")

In [22]:
from itertools import product

# (onein, (ptdiff, ctdiff))
goodtrails = [None]*16
rejected = []
for _ptdiff in product(range(64), repeat=4):
        
    ptdiff = 0
    for a in _ptdiff:
        ptdiff |= 1 << a
    ctdiff, prob = encryptblk(ptdiff, 12)

    ct_recovered = getctrecovered(ctdiff)
    onein = 1/prob
    for k in ct_recovered:
        kt = goodtrails[k]
        if kt is None or kt[0] > onein:
            goodtrails[k] = (onein, (ptdiff, ctdiff))
            if kt is not None: rejected.append(kt[1][0])

    #print(format(j, "02d"), "".join(".1"[i] for i in tobits(ctdiff, 64)), 1/trailprobability(trail))

In [24]:
[onein for onein, (ptdiff, ctdiff) in goodtrails]

[295147905.1793528,
 9007199.254740993,
 10613340.528852712,
 967140.6556917033,
 9007199.254740993,
 274877.906944,
 1093140.3895531585,
 57646.07523034235,
 15283210.361547904,
 910950.3246276322,
 549575.0201670253,
 97812.54631390658,
 967140.6556917033,
 57646.07523034235,
 117375.05557668791,
 12089.258196146291]

In [21]:
[onein for onein, (ptdiff, ctdiff) in goodtrails]

[175921.86044416,
 17179.869184,
 22239.9981598543,
 4503.599627370496,
 17179.869184,
 1677.7216,
 4886.718345671111,
 687.19476736,
 32025.597350190194,
 4886.718345671111,
 4048.668109456143,
 1281.0238940076079,
 4503.599627370496,
 687.19476736,
 1281.0238940076079,
 281.474976710656]

In [90]:
printtrail(goodtrails[0][1])

...1...........1................................................
................................................1..1............
............1...................................................
...1...............................................1............
................................................1...........1...
............1..1............................................1..1
...1...........1................................................
................................................1..1............
............1...................................................
...1...............................................1............
................................................1...........1...
............1..1............................................1..1
